<a href="https://colab.research.google.com/github/kidujm/data_mini_project/blob/main/%EC%9C%A0%EC%A0%95%EB%AF%BC_0210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 해야할 것
1. 행정도 코드 표준화
- (점포 데이터와 서울 데이터는 코드가 8자리인 반면, 경기도 데이터는 10자리 ) > 경기도 코드 앞자리 8자리 추출
2. 시점 및 칼럼 정규화
- 서울은 분기별,경기도는 월별 데이터이므로 분석 시점을 일치시키거나 최신 시점으로 필터링해야함
3. 데이터 결합



In [ ]:
import pandas as pd

# 파일 3개를 불러옵니다.
df_store = pd.read_csv('꿀빵떡1_최종.csv')
df_seoul = pd.read_csv('서울시 상권분석서비스(길단위인구-행정동).csv', encoding='CP949')
df_gyeonggi = pd.read_csv('경기도df_preprocessed.csv')

In [ ]:
# 1. 10자리 코드에서 앞 8자리만 가져오기
df_gyeonggi['행정동코드'] = df_gyeonggi['행정동_코드'].astype(str).str[:8].astype(int)

# 2. 가장 최근 날짜(2025년 3월) 데이터만 선택하기
df_gyeonggi_latest = df_gyeonggi[df_gyeonggi['기준_연월'] == 202503].copy()

# 3. 시간대별로 쪼개진 인구를 동네별로 다 더하기
df_gyeonggi_sum = df_gyeonggi_latest.groupby('행정동코드')['유동인구_수'].sum().reset_index()

# 4. 알아보기 쉽게 이름 바꾸기
df_gyeonggi_sum.columns = ['행정동코드', '유동인구']

In [ ]:
# 1. 서울의 가장 최근 데이터(2025년 3분기) 선택
df_seoul_latest = df_seoul[df_seoul['기준_년분기_코드'] == 20253].copy()

# 2. 필요한 칸(코드, 인구수)만 남기기
df_seoul_sum = df_seoul_latest[['행정동_코드', '총_유동인구_수']].copy()
df_seoul_sum.columns = ['행정동코드', '유동인구']

# 3. 서울과 경기도 인구 데이터를 하나로 합치기
df_pop_total = pd.concat([df_seoul_sum, df_gyeonggi_sum], axis=0)

In [ ]:
# '행정동코드'가 같은 것끼리 옆으로 결합합니다.
df_merged = pd.merge(df_store, df_pop_total, on='행정동코드', how='left')

# 우리가 분석하려는 '투다리'와 '역전할머니' 매장만 딱 골라냅니다.
df_final = df_merged[df_merged['brand'].isin(['투다리', '역전할머니'])].copy()

### 여기서 이제 구할 수 있는 것
- 유동인구가 많은 곳에 주로 입점한 브랜드가 어디인지
- 긴구가 적은 곳에서도 리뷰가 많이 발생한 브랜드가 어디인지

In [ ]:
# 파일 이름을 'merged_brand_analysis.csv'로 정합니다.
file_name = 'merged_brand_analysis.csv'
df_final.to_csv(file_name, index=False, encoding='utf-8-sig')

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   상가업소번호         730 non-null    object 
 1   brand          730 non-null    object 
 2   상호명            730 non-null    object 
 3   상권업종대분류코드      730 non-null    object 
 4   상권업종대분류명       730 non-null    object 
 5   상권업종중분류코드      730 non-null    object 
 6   상권업종중분류명       730 non-null    object 
 7   상권업종소분류코드      730 non-null    object 
 8   상권업종소분류명       730 non-null    object 
 9   표준산업분류코드       730 non-null    object 
 10  표준산업분류명        730 non-null    object 
 11  시도코드           730 non-null    int64  
 12  시도명            730 non-null    object 
 13  시군구코드          730 non-null    int64  
 14  시군구명           730 non-null    object 
 15  행정동코드          730 non-null    int64  
 16  행정동명           730 non-null    object 
 17  법정동코드          730 non-null    int64  
 18  법정동명      

In [ ]:
# 1. 빈 값(Null) 확인하기
print(df_final.isnull().sum())

상가업소번호            0
brand             0
상호명               0
상권업종대분류코드         0
상권업종대분류명          0
상권업종중분류코드         0
상권업종중분류명          0
상권업종소분류코드         0
상권업종소분류명          0
표준산업분류코드          0
표준산업분류명           0
시도코드              0
시도명               0
시군구코드             0
시군구명              0
행정동코드             0
행정동명              0
법정동코드             0
법정동명              0
지번코드              0
지번본번지             0
지번주소              0
도로명코드             0
도로명               0
도로명주소             0
경도                0
위도                0
place_id          0
월간 검색량            0
블로그누적발행           0
방문자사진수            0
역명                0
역_거리              0
역_거리(max1500)     0
추정_개업일            0
방문자리뷰수            0
블로그리뷰수            0
유동인구             87
dtype: int64


In [ ]:
# 2. '유동인구'의 빈 값을 0으로 채우기
# 인구 정보가 없는 곳은 0명으로 처리하여 계산 오류를 방지합니다.
df_final['유동인구'] = df_final['유동인구'].fillna(0)

In [ ]:
# 3. 리뷰수, 검색량 등 다른 숫자 칸의 빈 값도 0으로 채우기
cols = ['월간 검색량', '블로그누적발행', '방문자사진수', '방문자리뷰수', '블로그리뷰수']
df_final[cols] = df_final[cols].fillna(0)

In [ ]:
# 4. 중복된 데이터(줄) 삭제하기
# 똑같은 데이터가 여러 번 들어있는 경우 하나만 남기고 지웁니다.
df_final = df_final.drop_duplicates()

In [ ]:
# 5. 소수점으로 표시된 인구수를 보기 좋게 정수(1, 2, 3...)로 바꾸기
df_final['유동인구'] = df_final['유동인구'].astype(int)

In [ ]:
# 6. 깨끗해진 데이터를 새 파일로 저장하기
df_final.to_csv('cleaned_brand_analysis.csv', index=False, encoding='utf-8-sig')

print("전처리가 완료된 'cleaned_brand_analysis.csv' 파일이 저장되었습니다.")

전처리가 완료된 'cleaned_brand_analysis.csv' 파일이 저장되었습니다.
